# Voxel Playground

In [3]:
# Import dependencies
from open3d import read_point_cloud
from open3d import voxel_down_sample
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# Read ply files to point cloud format
prostate = read_point_cloud('../0_data/prostate-no-nodes/Mesh_PtNum-1455-PTVHD.ply')
bladder = read_point_cloud('../0_data/prostate-no-nodes/Mesh_PtNum-1455-Bladder.ply')
feml = read_point_cloud('../0_data/prostate-no-nodes/Mesh_PtNum-1455-FemoralHeadL.ply')
femr = read_point_cloud('../0_data/prostate-no-nodes/Mesh_PtNum-1455-FemoralHeadR.ply')
rectum = read_point_cloud('../0_data/prostate-no-nodes/Mesh_PtNum-1455-Rectum.ply')

# Convert point cloud format to cartesian coordinates
prostate = np.asarray(prostate.points)
bladder = np.asarray(bladder.points)
feml = np.asarray(feml.points)
femr = np.asarray(femr.points)
rectum = np.asarray(rectum.points)

# Add dimension to signify that prostate is target
prostate=np.insert(prostate, 3, 1, axis=1)
bladder=np.insert(bladder, 3, 0, axis=1)
feml=np.insert(feml, 3, 0, axis=1)
femr=np.insert(femr, 3, 0, axis=1)
rectum=np.insert(rectum, 3, 0, axis=1)

# Combine them 
combined = np.concatenate((prostate,bladder,feml,femr,rectum))

In [5]:
combined.shape

(8483, 4)

In [6]:
combined[:,0:3].shape

(8483, 3)

In [7]:
sum(combined[:,3]==1)

1337

In [8]:
# farthest point calculation
def calc_distances(p0, points):
    return ((p0 - points)**2).sum(axis=1)

def downsample(pts, K):
    farthest_pts = np.zeros((K, 3))
    farthest_pts[0] = pts[np.random.randint(len(pts))]
    distances = calc_distances(farthest_pts[0], pts)
    for i in range(1, K):
        farthest_pts[i] = pts[np.argmax(distances)]
        distances = np.minimum(distances, calc_distances(farthest_pts[i], pts))
    return farthest_pts

In [9]:
#points = combined[:, 0:3]
points = downsample(combined[:, 0:3], 1024)

In [10]:
points.shape

(1024, 3)

In [12]:
# re-attach 'l' to downsampled scan 
#patient = np.concatenate((points, combined[:,3]), axis=1)

In [13]:
print(prostate.shape)
print(bladder.shape)
print(feml.shape)
print(femr.shape)
print(rectum.shape)

(1337, 4)
(1576, 4)
(2280, 4)
(2233, 4)
(1057, 4)


In [14]:
pt1455[0] = np.expand_dims(prostate, axis=0)
pt1455[0].shape

NameError: name 'pt1455' is not defined

In [15]:
h,w,d=16,16,16
prostate_grid = VoxelGrid(prostate, x_y_z=[h,w,d])
prostate_voxel = voxel_grid.vector
prostate_grid.plot()

NameError: name 'voxel_grid' is not defined

In [16]:
prostate_voxel.shape

NameError: name 'prostate_voxel' is not defined

In [17]:
# Add dimension to signify that prostate is target
from matplotlib.pyplot import cm

def add_rgb_dimention(array, color):
    scaler_map = cm.ScalarMappable(cmap=color)
    array = scaler_map.to_rgba(array)[:, : -1]
    return array

placeholder = np.ndarray((1, (h*w*d), 3))
prostate_voxel = prostate_voxel.ravel()
placeholder[0] = add_rgb_dimention(prostate_voxel, color="Oranges")

placeholder.shape

placeholder.reshape(1,16,16,16,3)

NameError: name 'prostate_voxel' is not defined

In [ ]:
placeholder.shape

In [ ]:
# plot point cloud 
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
	
def plot_ply(file):
    fig = plt.figure()
    ax = fig.add_subplot(111,projection='3d')
    ax.set_ylim(-300,300)
    ax.set_xlim(-300,300)
    ax.set_zlim(-300,300)
    x=file[:,0]
    y=file[:,1]
    z=file[:,2]
    ax.scatter(x, y, z, marker='.', zdir='z')
    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')
    ax.set_zlabel('Z Label')
    plt.show()

In [ ]:
plot_ply(prostate)
print(prostate)
print('Cloud shape:', prostate.shape)

In [ ]:
combined = np.concatenate((prostate,bladder,feml,femr,rectum))
plot_ply(combined)
print(combined)
print('Cloud shape:', combined.shape)

In [ ]:
from matplotlib.pyplot import cm

def add_rgb_dimention(array, color):
    scaler_map = cm.ScalarMappable(cmap=color)
    array = scaler_map.to_rgba(array)[:, : -1]
    return array


In [ ]:
combined.shape

In [ ]:
"""
VoxelGrid Class
"""
import numpy as np
from matplotlib import pyplot as plt
#from ..plot import plot_voxelgrid

class VoxelGrid(object):
    def __init__(self, points, x_y_z=[1, 1, 1], bb_cuboid=True, build=True):
        """
        Parameters
        ----------         
        points: (N,3) ndarray
                The point cloud from which we want to construct the VoxelGrid.
                Where N is the number of points in the point cloud and the second
                dimension represents the x, y and z coordinates of each point.
        
        x_y_z:  list
                The segments in which each axis will be divided.
                x_y_z[0]: x axis 
                x_y_z[1]: y axis 
                x_y_z[2]: z axis

        bb_cuboid(Optional): bool
                If True(Default):   
                    The bounding box of the point cloud will be adjusted
                    in order to have all the dimensions of equal length.                
                If False:
                    The bounding box is allowed to have dimensions of different sizes.
        """
        self.points = points
        xyzmin = np.min(points, axis=0) - 0.001
        xyzmax = np.max(points, axis=0) + 0.001

        if bb_cuboid:
            #: adjust to obtain a  minimum bounding box with all sides of equal length 
            diff = max(xyzmax-xyzmin) - (xyzmax-xyzmin)
            xyzmin = xyzmin - diff / 2
            xyzmax = xyzmax + diff / 2
        
        self.xyzmin = xyzmin
        self.xyzmax = xyzmax
        segments = []
        shape = []

        for i in range(3):
            # note the +1 in num 
            if type(x_y_z[i]) is not int:
                raise TypeError("x_y_z[{}] must be int".format(i))
            s, step = np.linspace(xyzmin[i], xyzmax[i], num=(x_y_z[i] + 1), retstep=True)
            segments.append(s)
            shape.append(step)
        self.segments = segments
        self.shape = shape
        self.n_voxels = x_y_z[0] * x_y_z[1] * x_y_z[2]
        self.n_x = x_y_z[0]
        self.n_y = x_y_z[1]
        self.n_z = x_y_z[2]
        self.id = "{},{},{}-{}".format(x_y_z[0], x_y_z[1], x_y_z[2], bb_cuboid)
        if build:
            self.build()

    def build(self):
        structure = np.zeros((len(self.points), 4), dtype=int)
        structure[:,0] = np.searchsorted(self.segments[0], self.points[:,0]) - 1
        structure[:,1] = np.searchsorted(self.segments[1], self.points[:,1]) - 1
        structure[:,2] = np.searchsorted(self.segments[2], self.points[:,2]) - 1
        # i = ((y * n_x) + x) + (z * (n_x * n_y))
        structure[:,3] = ((structure[:,1] * self.n_x) + structure[:,0]) + (structure[:,2] * (self.n_x * self.n_y))
        self.structure = structure
        vector = np.zeros(self.n_voxels)
        count = np.bincount(self.structure[:,3])
        vector[:len(count)] = count
        self.vector = vector.reshape(self.n_z, self.n_y, self.n_x)
 
    def plot(self, d=2, cmap="Oranges", axis=False):
        if d == 2:
            fig, axes= plt.subplots(int(np.ceil(self.n_z / 4)), 4, figsize=(8,8))
            plt.tight_layout()
            
            for i,ax in enumerate(axes.flat):
                if i >= len(self.vector):
                    break
                im = ax.imshow(self.vector[i], cmap=cmap, interpolation="none")
                ax.set_title("Level " + str(i))
            
            fig.subplots_adjust(right=0.8)
            cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
            cbar = fig.colorbar(im, cax=cbar_ax)
            cbar.set_label('NUMBER OF POINTS IN VOXEL')
            fig.savefig('subplottotal.png')
        elif d == 3:
            return plot_voxelgrid(self, cmap=cmap, axis=axis)


In [ ]:
###################
#specify dimensions
h,w,d = 32,32,32
###################
voxel_grid = VoxelGrid(combined, x_y_z=[h,w,d])
voxel_array = voxel_grid.vector
voxel_grid.plot()

import matplotlib.pyplot as plt

# This import registers the 3D projection, but is otherwise unused.
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import

# and plot everything
fig = plt.figure()
ax = fig.gca(projection='3d')
#ax.set_xlim3d(0, 102)
#ax.set_ylim3d(0, 85)
#ax.set_zlim3d(0, 120)
ax.voxels(voxel_array, edgecolor='k')
plt.savefig('foo.png')
plt.show()

In [ ]:
voxel_array.shape

In [25]:
import os
import nibabel as nib


In [27]:
aff = img.get_affine()
real_pt = nib.affines.apply_affine(aff, [22, 34, 12])
real_pt

NameError: name 'img' is not defined